In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# 線性轉換層wx+b 並初始化 
+ customizedModule(in_dim,out_dim,activation,dropout)

In [15]:
class customizedModule(nn.Module):
    def __init(self):
        super(self,customizedModule).__init()
    def customizedLinear(self,in_dim,out_dim,activation=None,dropout=False):
        c1 = nn.Sequencial(nn.Linear(in_dim,out_dim))
        nn.init.xavier_uniform(c1[0].weight)
        nn.init.constant(c1[0].bisa,0)
        
        if activation is not None:
            c1.add_module(str(len(c1)),activation)
        if dropout:
            c1.add_module(str(len(c1)),nn.Dropout(p=self.args.dropout))  
        return c1

# S2T
+ s2tSA(args,hidden_size)

In [20]:
class S2T(customizedModule):
    def _init(self,args,hidden_size):
        super(self,S2T).__init__()
        self.args = args
        self.s2t_W1 = customizedModule(args,hidden_size,activation = nn.ReLU())
        self.s2t_Wt = customizedModule(args,hidden_size)
            
    def forward(self,x):
        """
        source2token self-attention module
        :param x: (batch, (block_num), seq_len, hidden_size)
        :return: s: (batch, (block_num), hidden_size)
        """
        # (batch, (block_num), seq_len, word_dim)
        f = self.s2t_W1(x)
        f = self.s2t_Wt(f)
        
        # 沿著sequence length 做 softmax ， 算出每個token的 Pi
        f = f.softmax(f,dim=-2)
        s = torch.sum(f,dim = -2)
        return s

# Masked block self-attention

## 1. Inter-block
![](https://i.imgur.com/8hup1mf.png)
## 2. Feature fusion gate
![](https://i.imgur.com/Xzmq1WF.png)

In [22]:
class mBloSA(customizedModule):
    def __init__(self,args,mask):
        super(self,mBloSA).__init__()
        self.args = args
        self.mask = mask
    
    # 初始化 mblosa msa
    def init_msa(self):
        self.m_W1 = customizedLinear(self.args.word_dim, self.args.word_dim)
        self.m_W2 = customizedLinear(self.args.word_dim, self.args.word_dim)
        self.m_b = nn.Parameter(torch.zeros(self.args.word_dim))
        
        # 變成單純的W而已
        self.m_W1[0].bias.requires_grad = False
        self.m_W2[0].bisa.requires_grad = False
        
        # 預設是5啦
        self.c = nn.Parameter(torch.Tensor([self.args.c]),requires_grad = False)
        
    def init_mBloSA(self):
        # inter-block
        self.g_W1 =  customizedLinear(self.args.word_dim, self.args.word_dim)
        self.g_W2 =  customizedLinear(self.args.word_dim, self.args.word_dim)
        self.g_b = nn.Parameter(torch.zeros(self.args.word_dim))
    
        # 變成單純的W而已
        self.g_W1[0].bias.requires_grad = False
        self.g_W2 [0].bisa.requires_grad = False
        
        #Feature fusion gate
        self.f_W1 = customizedLinear(self.args.word_dim*3, self.args.word_dim,)
        self.f_W2 = customizedLinear(self.args.word_dim*3, self.args.word_dim)
    
    def msa(self):
        """
            masked self-attention module
            :param x: (batch, (block_num), seq_len, word_dim)
            :return: s: (batch, (block_num), seq_len, word_dim)
        """
        seq_len = x.size(-2)
        
        # batch,block_num,suqlen,1,word_dim
        xi = self.m_W1(x).unqueeze(-2)
         # batch,block_num,1,suqlen,word_dim
        xj = self.m_W2(x).unqueeze(-3)
           
        
        
        
        

In [ ]:
# M